In [1]:
from configparser import ConfigParser
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

In [2]:
input = pd.read_csv('artists.csv', header = None)
artist_ids = []
for link in input[0]:
    
    artist_ids.append(link[32:54])

artist_ids

['4tZwfgrHOc3mvqYlEYSvVi', '2CIMQHirSU0MQqyYHq0eOx', '6nxWCVXbOlEVRexSbLsTer']

In [41]:
albums = []
album_track_ids = []

for artist_id in artist_ids:
    
    album_ids = []
    artist_uri = 'spotify:artist:' + artist_id

    # Add album ids to album_ids list
    results = sp.artist_albums(artist_uri, album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    for album in albums:
        album_ids.append(album['id'])


for id in album_ids:

    track_ids = []

    results = sp.album_tracks(id)
    tracks = results['items']
    for track in tracks:
        track_ids.append(track['id'])

    album_track_ids.append(track_ids)


assert len(album_track_ids) == len(albums)

len(albums)

71

In [4]:
columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
headings = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'id', 'duration_ms', 'time_signature','album_title', 'album_artist']
data = pd.DataFrame(columns=headings)


# Create DataFrames of features from track ids
i = 0
for album in album_track_ids:

    features = []
    for track in album:
        results = sp.audio_features(track)
        features.append(results[0])
    features_matrix = pd.DataFrame.from_records(features)

    # Remove unneeded columns
    features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)

    feature_lists = []
    for column in features_matrix:
        feature_lists.append(list(features_matrix[column]))

    # Add album title
    title = albums[i]['name']
    feature_lists.append(title)
    # Add artist name
    name = albums[i]['artists'][0]['name']
    feature_lists.append(name)

    # Add album as a row to data df
  
    data.loc[i] = feature_lists
    i += 1

data.drop_duplicates(subset=['album_title', 'album_artist'])

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,album_title,album_artist
0,"[0.788, 0.712, 0.74, 0.702, 0.775, 0.832, 0.29...","[0.616, 0.5, 0.682, 0.297, 0.585, 0.659, 0.412...","[9, 9, 9, 10, 10, 10, 6, 6, 8, 4, 0, 6, 11]","[-10.671, -11.948, -10.42, -14.116, -9.516, -7...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]","[0.0337, 0.0338, 0.0504, 0.0317, 0.0271, 0.057...","[0.00286, 0.825, 0.0692, 0.894, 0.0422, 0.0839...","[0.549, 0.314, 0.53, 0.485, 0.619, 0.00114, 0....","[0.0952, 0.106, 0.381, 0.103, 0.077, 0.0753, 0...","[0.696, 0.682, 0.41, 0.0676, 0.518, 0.674, 0.1...","[119.037, 90.039, 113.201, 110.116, 109.942, 1...","[0dEIca2nhcxDUV8C5QkPYb, 3ctALmweZBapfBdFiIVpj...","[275387, 322147, 544627, 228507, 337560, 35389...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]",Random Access Memories,Daft Punk
1,"[0.717, 0.723, 0.707, 0.635, 0.679, 0.573, 0.6...","[0.382, 0.389, 0.515, 0.476, 0.377, 0.442, 0.3...","[9, 9, 6, 6, 2, 7, 9, 9]","[-13.752, -13.204, -12.299, -13.942, -14.144, ...","[1, 1, 0, 0, 1, 1, 1, 1]","[0.64, 0.8, 0.772, 0.446, 0.553, 0.664, 0.768,...","[0.506, 0.424, 0.359, 0.297, 0.313, 0.666, 0.7...","[0, 0, 0, 0, 0, 0, 0, 0]","[0.242, 0.183, 0.237, 0.181, 0.163, 0.23, 0.25...","[0.622, 0.624, 0.654, 0.6, 0.586, 0.55, 0.666,...","[116.632, 118.413, 115.171, 116.154, 116.597, ...","[4l2bO887Yauqos4LwkCwBm, 3K6HXNtxo1A2eCthCzrwa...","[473507, 458493, 453200, 422827, 457813, 51646...","[4, 3, 4, 4, 4, 3, 4, 4]",Daft Punk | Random Access Memories | The Colla...,Daft Punk
2,"[0.518, 0.373, 0.634, 0.741, 0.431, 0.595, 0.6...","[0.527, 0.841, 0.798, 0.806, 0.955, 0.605, 0.8...","[8, 4, 1, 1, 7, 6, 7, 9, 11, 7, 11, 10, 7, 3, 1]","[-7.684, -6.475, -6.86, -6.443, -5.601, -6.746...","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0687, 0.0614, 0.0607, 0.0371, 0.0361, 0.031...","[0.0569, 0.00141, 0.0283, 0.00531, 0.0175, 0.0...","[0.00263, 0.642, 0.546, 0.783, 0.693, 0.802, 0...","[0.107, 0.342, 0.1, 0.133, 0.164, 0.181, 0.423...","[0.213, 0.164, 0.494, 0.248, 0.282, 0.163, 0.2...","[96.995, 89.926, 125.011, 126.967, 106.011, 13...","[4lteJuSjb9Jt9W1W7PIU2U, 66uVqkmHAc0MBUzoPhIyp...","[262240, 234987, 267787, 334347, 291507, 27526...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4]",TRON: Legacy Reconfigured,Daft Punk
3,"[0.518, 0.373, 0.634, 0.741, 0.431, 0.595, 0.6...","[0.527, 0.841, 0.798, 0.806, 0.955, 0.605, 0.8...","[8, 4, 1, 1, 7, 6, 7, 9, 11, 7, 10, 10, 7, 3, 1]","[-7.684, -6.475, -6.86, -6.443, -5.601, -6.746...","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]","[0.0687, 0.0614, 0.0607, 0.0371, 0.0361, 0.031...","[0.0569, 0.00141, 0.0283, 0.00531, 0.0175, 0.0...","[0.00263, 0.642, 0.546, 0.783, 0.693, 0.802, 0...","[0.107, 0.342, 0.1, 0.133, 0.164, 0.181, 0.423...","[0.213, 0.164, 0.494, 0.248, 0.282, 0.163, 0.2...","[96.995, 89.926, 125.011, 126.967, 106.011, 13...","[388Lvca2KrUQE9H7ncsPaR, 0nOQG4M0I1ZQQZwObJyyp...","[262240, 234987, 267787, 334347, 291507, 27526...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]",TRON: Legacy Reconfigured (Australian Version),Daft Punk
4,"[0.125, 0.555, 0.546, 0.578, 0.394, 0.0671, 0....","[0.0472, 0.524, 0.767, 0.531, 0.0518, 0.423, 0...","[1, 1, 0, 1, 5, 3, 4, 9, 0, 1, 0, 10, 1, 11, 1...","[-14.881, -13.401, -17.622, -14.7, -23.923, -1...","[1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[0.0418, 0.0381, 0.0394, 0.0394, 0.0373, 0.042...","[0.903, 0.266, 0.901, 0.448, 0.649, 0.0169, 0....","[0.969, 0.122, 0.909, 0.548, 0.938, 0.984, 0.8...","[0.126, 0.341, 0.22, 0.11, 0.117, 0.0713, 0.09...","[0.0375, 0.0645, 0.0261, 0.0803, 0.0331, 0.093...","[74.53, 102.042, 105.99, 104.015, 96.016, 66.8...","[2nyv3x8gaHrVCO2lY7N0d7, 6X8SkAJoJykm2gpMlBoJT...","[148013, 96747, 95227, 157707, 122987, 93227, ...","[3, 4, 4, 4, 4, 5, 4, 4, 4, 5, 0, 4, 4, 4, 3, ...",TRON: Legacy (Original Soundtrack),Daft Punk
5,"[0.104, 0.548, 0.543, 0.577, 0.384, 0.334, 0.4...","[0.0818, 0.496, 0.775, 0.539, 0.0475, 0.423, 0...

In [5]:
data.to_csv('./data.csv')

In [6]:
# Live
sp.audio_features('2RpXUwCytDFqZSHl6kIquB')

[{'danceability': 0.26,
  'energy': 0.99,
  'key': 7,
  'loudness': -3.498,
  'mode': 1,
  'speechiness': 0.128,
  'acousticness': 0.00202,
  'instrumentalness': 0.00416,
  'liveness': 0.779,
  'valence': 0.0885,
  'tempo': 128.29,
  'type': 'audio_features',
  'id': '2RpXUwCytDFqZSHl6kIquB',
  'uri': 'spotify:track:2RpXUwCytDFqZSHl6kIquB',
  'track_href': 'https://api.spotify.com/v1/tracks/2RpXUwCytDFqZSHl6kIquB',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2RpXUwCytDFqZSHl6kIquB',
  'duration_ms': 349733,
  'time_signature': 4}]

In [7]:
# Remastered
sp.audio_features('0xzhryP1AoHUazYdJ5rj3B')

[{'danceability': 0.46,
  'energy': 0.922,
  'key': 2,
  'loudness': -7.388,
  'mode': 1,
  'speechiness': 0.264,
  'acousticness': 0.0449,
  'instrumentalness': 0.000252,
  'liveness': 0.34,
  'valence': 0.328,
  'tempo': 119.961,
  'type': 'audio_features',
  'id': '0xzhryP1AoHUazYdJ5rj3B',
  'uri': 'spotify:track:0xzhryP1AoHUazYdJ5rj3B',
  'track_href': 'https://api.spotify.com/v1/tracks/0xzhryP1AoHUazYdJ5rj3B',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0xzhryP1AoHUazYdJ5rj3B',
  'duration_ms': 311027,
  'time_signature': 4}]

In [8]:
# Non-remastered (original)
sp.audio_features('4BB3Itr3YETOKFURnC28Ld')

[{'danceability': 0.472,
  'energy': 0.859,
  'key': 2,
  'loudness': -11.866,
  'mode': 1,
  'speechiness': 0.181,
  'acousticness': 0.0793,
  'instrumentalness': 0.000202,
  'liveness': 0.645,
  'valence': 0.322,
  'tempo': 119.989,
  'type': 'audio_features',
  'id': '4BB3Itr3YETOKFURnC28Ld',
  'uri': 'spotify:track:4BB3Itr3YETOKFURnC28Ld',
  'track_href': 'https://api.spotify.com/v1/tracks/4BB3Itr3YETOKFURnC28Ld',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4BB3Itr3YETOKFURnC28Ld',
  'duration_ms': 310667,
  'time_signature': 4}]

In [53]:
# album_ids = ['4y7T04fej3MJ8tAe2FeKD5']
# album_track_ids = []

# for id in album_ids:

#     track_ids = []

#     results = sp.album_tracks(id)
#     tracks = results['items']
#     for track in tracks:
#         track_ids.append(track['id'])

#     album_track_ids.append(track_ids)

section_track_ids = album_track_ids[:33]
section_data = []

# print(section_track_ids)

for album in section_track_ids:
    for track in album:

        section_data.append(sp.audio_analysis(track)['sections'])

print(section_data)


[[{'start': 0.0, 'duration': 18.9774, 'confidence': 1.0, 'loudness': -8.062, 'tempo': 119.052, 'tempo_confidence': 0.403, 'key': 9, 'key_confidence': 0.656, 'mode': 0, 'mode_confidence': 0.395, 'time_signature': 4, 'time_signature_confidence': 1.0}, {'start': 18.9774, 'duration': 34.77193, 'confidence': 1.0, 'loudness': -11.683, 'tempo': 119.029, 'tempo_confidence': 0.853, 'key': 9, 'key_confidence': 0.502, 'mode': 0, 'mode_confidence': 0.484, 'time_signature': 4, 'time_signature_confidence': 1.0}, {'start': 53.74933, 'duration': 28.22747, 'confidence': 0.723, 'loudness': -9.778, 'tempo': 119.047, 'tempo_confidence': 0.754, 'key': 0, 'key_confidence': 0.303, 'mode': 1, 'mode_confidence': 0.383, 'time_signature': 4, 'time_signature_confidence': 1.0}, {'start': 81.9768, 'duration': 32.76324, 'confidence': 0.463, 'loudness': -10.56, 'tempo': 119.053, 'tempo_confidence': 0.743, 'key': 7, 'key_confidence': 0.204, 'mode': 1, 'mode_confidence': 0.489, 'time_signature': 4, 'time_signature_conf